In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import re

# specify GPU
device = torch.device("cuda")

In [158]:
data = pd.read_csv('re_dataset.csv', encoding = 'ISO-8859-1')
data

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13164,USER jangan asal ngomong ndasmu. congor lu yg ...,1,1,1,0,0,0,1,0,0,1,0,0
13165,USER Kasur mana enak kunyuk',0,1,0,0,0,0,0,0,0,0,0,0
13166,USER Hati hati bisu :( .g\n\nlagi bosan huft \...,0,0,0,0,0,0,0,0,0,0,0,0
13167,USER USER USER USER Bom yang real mudah terdet...,0,0,0,0,0,0,0,0,0,0,0,0


In [159]:
data['label'] = 0

data.loc[data.HS == 0, 'label'] = 0
data.loc[data.HS_Weak == 1, 'label'] = 1
data.loc[data.HS_Moderate == 1, 'label'] = 2
data.loc[data.HS_Strong == 1, 'label'] = 3

data = data[['Tweet', 'label']]
data

,Tweet,label
0,- disaat semua cowok berusaha melacak perhatia...,1
1,RT USER: USER siapa yang telat ngasih tau elu?...,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0
4,USER USER Kaum cebong kapir udah keliatan dong...,2
...,...,...
13164,USER jangan asal ngomong ndasmu. congor lu yg ...,1
13165,USER Kasur mana enak kunyuk',0
13166,USER Hati hati bisu :( .g\n\nlagi bosan huft \...,0
13167,USER USER USER USER Bom yang real mudah terdet...,0


In [160]:
data['label'].value_counts()

0    7608
1    3383
2    1705
3     473
Name: label, dtype: int64

In [161]:
alay_dict = pd.read_csv('new_kamusalay.csv', encoding='latin-1', header=None)
alay_dict = alay_dict.rename(columns={0: 'original', 
                                      1: 'replacement'})

Noise Removal

In [162]:
def lowercase(text):
    return text.lower()

In [163]:
def remove_unnecessary_char(text):
    text = re.sub(r'\\x[A-Za-z0-9./]+',r' ',text)
    text = re.sub(r'\n',r' ',text) # Remove every '\n'
    text = re.sub(r'rt',r' ',text) # Remove every retweet symbol
    text = re.sub(r'user',r' ',text) # Remove every username
    text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',r' ',text) # Remove every URL
    text = re.sub(r'  +', r' ', text) # Remove extra spaces
    text = re.sub(r'[^0-9a-zA-Z]+', r' ', text) 
    text = re.sub(r'[0-9\-]+', r' ', text) 
    return text

In [164]:
def remove_stopword(text):
    text = ' '.join(['' if word in stopword_dict.stopword.values else word for word in text.split(' ')])
    text = re.sub('  +', ' ', text) # Remove extra spaces
    text = text.strip()
    return text

In [165]:
def noiseremove(text):
    text = lowercase(text) # 1
    text = remove_unnecessary_char(text) # 2

    return text

In [166]:
data['Tweet'] = data['Tweet'].apply(noiseremove)
data.head(10)

C:\Users\Hanun\AppData\Local\Temp\ipykernel_3468\601464678.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Tweet'] = data['Tweet'].apply(noiseremove)


,Tweet,label
0,disaat semua cowok berusaha melacak perhatian...,1
1,siapa yang telat ngasih tau elu edan sarap gu...,0
2,kadang aku berfikir kenapa aku tetap percaya...,0
3,aku itu aku n nku tau matamu sipit tapi dilia...,0
4,kaum cebong kapir udah keliatan dongoknya dar...,2
5,ya bani taplak dkk,2
6,deklarasi pilkada aman dan anti hoax warga d...,0
7,gue baru aja kelar re watch aldnoah zero palin...,0
8,nah admin belanja satu lagi po terbaik nak mak...,0
9,enak lg klo smbil ngewe,0


Normalization

In [167]:
alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))
def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])

In [168]:
def delete_white_space(text):
    text = text.strip()
    text = text.replace("     ", " ")
    text = text.replace("    ", " ")
    text = text.replace("   ", " ")
    text = text.replace("  ", " ")
    return text

In [169]:
def normaliza(text):
    text = normalize_alay(text)
    text = delete_white_space(text)
    return text

In [170]:
data['Tweet'] = data['Tweet'].apply(normaliza)
data.head(10)

C:\Users\Hanun\AppData\Local\Temp\ipykernel_3468\1571131175.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Tweet'] = data['Tweet'].apply(normaliza)


,Tweet,label
0,di saat semua cowok berusaha melacak perhatian...,1
1,siapa yang telat memberi tau kamu edan sarap g...,0
2,kadang aku berpikir kenapa aku tetap percaya p...,0
3,aku itu aku dan ku tau matamu sipit tapi dilih...,0
4,kaum cebong kafir sudah kelihatan dongoknya da...,2
5,ya bani taplak dan kawan kawan,2
6,deklarasi pilihan kepala daerah aman dan anti ...,0
7,gue baru saja selesai re watch aldnoah zero pa...,0
8,nah admin belanja satu lagi po terbaik nak mak...,0
9,enak lagi kalau sambil ngewe,0


In [171]:
data = data.replace(r'', np.nan, regex=True)
print("Sebelum drop nan : {}".format(len(data)))

data3 = data2.dropna(subset=['Tweet'])
print("Sebelum drop nan : {}".format(len(data3)))

Sebelum drop nan : 13169
Sebelum drop nan : 13123


In [172]:
data.to_csv('data.csv',index=False)

In [173]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(data['Tweet'],data['label'],stratify=data['label'],test_size=0.20)

print("Jumlah train data : {}".format(len(y_train)))
print("Jumlah test data : {}".format(len(y_test)))

Jumlah train data : 10535
Jumlah test data : 2634


In [174]:
df_train = pd.DataFrame([y_train, X_train]).transpose()
df_train.to_csv('data_train.txt', sep='\t', index=False, header=False)
df_train

,label,Tweet
11250,0,belajarlah dari babi ngepet kalau malam cuma p...
1004,1,prabowo subianto tak restui ketua gerindra jaw...
10285,0,ambang batas calon presiden dipermasalahkan se...
992,2,anda menghina islam muhammad tidak tahu budi t...
2496,0,ritual puasa tiap tanggal muharam adalah tradi...
...,...,...
12255,0,bangkai tiponya parah betul anjir dan tidak li...
10521,2,umat hindu dipermak tentara buddha radikal jad...
3328,2,yang mulai memaki cebong juga kaum codot memai...
6717,0,sudah kamu tanya aku jawab lah haha sudah itu ...


In [175]:
df_test = pd.DataFrame([y_test, X_test]).transpose()
df_test.to_csv('data_test.txt', sep='\t', index=False, header=False)
df_test

,label,Tweet
12493,0,apa kabar memek tembem
1927,0,temanku juga punya teman di cina yang sering m...
3557,3,yang beda paham harus di kuliti tangkap itu si...
3914,0,kom menteri hukum dan hak asasi manusia pernah...
10190,1,selama pemimpinnya dungu rakyat tidak kan mend...
...,...,...
9996,1,ejek bacot kamu ngentot nikahi makanya bukan m...
1449,1,di tunggu aksi bela kafir hari tanu
12005,0,propaganda agama selalu pemberontakan dipimpin...
6108,0,merong adalah bukan karena gendut atau kurus p...


In [183]:
!python eda_nlp-master/code/augment.py --input=data_train.txt --output=augmented_data_train.txt --num_aug=2 --alpha_sr=0.1 --alpha_rd=0 --alpha_ri=0 --alpha_rs=0

generated augmented sentences with eda for data_train.txt to augmented_data_train.txt with num_aug=2


In [188]:
dataTrain = pd.read_csv('augmented_data_train.txt', sep="\t", header=None)
dataTrain.columns = ["label", "Tweet"]

convert_dict = {'label': int,
                'Tweet': str
                }
 
dataTrain = dataTrain.astype(convert_dict)

In [190]:
dataTrain.to_csv('data_train.csv',index=False)

In [191]:
dataTest = pd.read_csv('data_test.txt', sep="\t", header=None)
dataTest.columns = ["label", "Tweet"]

convert_dict = {'label': int,
                'Tweet': str
                }
 
dataTest = dataTest.astype(convert_dict)

In [192]:
dataTest.to_csv('data_test.csv',index=False)

In [193]:
print("Data sebelum augmentasi {}".format(len(df_train)))
print("Data setelah augmentasi {}".format(len(dataTrain)))

Data sebelum augmentasi 10535
Data setelah augmentasi 31605
